In [41]:
from dotenv import load_dotenv
load_dotenv()

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from pprint import pprint
from dataclasses import dataclass
from typing import *
from pprint import pprint
import jsons

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [42]:
# set up and parsing of data
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

df = pd.read_csv('../resources/bubbleflexe-rv.csv', usecols=[1, 2], names=['bsides', 'tt'])
df = df.drop(labels=0, axis=0).reset_index(drop=True)
for c in df.columns:
    df[c] = df[c].apply(lambda x: str(x).split(';'))
# unify the responded songs
df['unified'] = df['bsides'] + df['tt']

In [43]:
# Utilities
def dictionaryFilter(d: dict, func) -> dict:
    '''Filter a dictionary d by items that result in true for func(k, v)'''
    return {k: v for k, v in d.items() if func(k, v)}

In [44]:
unique_bsides = set()
for i in df['bsides']:
    unique_bsides.update(i)

unique_tt = set()
for i in df['tt']:
    unique_tt.update(i)

unique_songs = unique_bsides.union(unique_tt)
unique_songs.remove('nan')

## Data Introspection

In [45]:
# TODO distribution of size of responses.

## Questions
1. For a given song, what is the frequency list of all other songs to it? -> Done
2. How to use the size of each individual response? -> **Not yet added**
3. What about normalizing for song popularity -> Using percents, Done

In [72]:
@dataclass
class Song:
    # todo make init based on name and artist make a spotify req to populate the object's name and artist
    def __init__(self, name: str, artist: str = 'Red Velvet', id: str = None, spotify_af: dict = None, tags: list = None):
        self.name = name
        self.artist = artist
        self.id = id or self.__get_track_id()
        self.spotify_af = spotify_af or self.__get_audio_analysis()
        self.tags = tags
    
    def __get_track_id(self):
        r = spotify.search(q=f'{self.name} artist:{self.artist}', type='track')
        return r['tracks']['items'][0]['id']
    
    def __get_audio_analysis(self):
        ft = spotify.audio_features(self.id)[0]
        delkeys = ['type', 'id', 'uri', 'track_href', 'analysis_url']
        [ft.pop(x) for x in delkeys]
        return ft

    # def __repr__(self):
    #     # TODO see if this is safe
    #     return f'{self.name} - {self.artist} - {str(self.tags)}'

    def __str__(self):
        return f'{self.name} - {self.artist} - [{str(self.tags)}]'

    # def __key(self):
    #     return (self.name) # TODO THIS IS ONLY DUE TO WORKING WITH RV CONTENT.
    #     # TODO find out a better way to make SongCollection like a dictionary but with regex matching for song name; make easier to extract from responses

    # def __hash__(self):
    #     # note that we are heavily breaking convention here
    #     return hash(self.__key())

    # def __eq__(self, o):
    #     if isinstance(o, Song):
    #         # if self.name == o.name and self.artist == o.artist:
    #         #     return True
    #         return self.__key() == o.__key()
    #     elif isinstance(o, str):
    #         # this is more useful for when there are multiple artists. we are only working with RV songs, so it's looser
    #         # if o == str(self)[:str(self).rfind('-')-1]:
    #         #     return True
    #         if o.lower() == self.name.lower():
    #             return True
    #     return False

# TODO transition into adjacency matrix
@dataclass
class SongCollection:
    def __init__(self, songs: List[Song], responses: pd.DataFrame, dig=2):
        self.songs = songs 
        self.responses = responses
        self.count_list = self.__get_count_list()
        self.song_count_list = {s: self.__get_song_count_list(s) for s in self.songs}
        self.percents_cache = self.__get_cache_dict(dig)
        
    def __get_cache_dict(self, dig):
        d = {}
        for s in self.songs:
            d[s] = {'inbound': self.__get_song_inbound_percent_list(s, dig), 
                    'outbound': self.__get_song_outbound_percent_list(s, dig)}
        return d
            
    def get(self, match) -> Song:
        return self.__get_handler(self.__get_by_name, match)

    def __get_handler(self, method, match):
        r = method(match)
        if len(r) > 1:
            raise Exception('too many songs matched')
        elif len(r) == 0:
            raise Exception('no matching Song found')
        return r[0]

    def __get_by_name(self, name):
        name = name.lower()
        r = [s for s in self.songs if s.name.lower() == name]
        # r = list(filter(lambda s: s.name.lower() == name, self.songs))
        return r

    def __get_by_eq(self, obj):
        r = list(filter(lambda s: s == obj, self.songs))
        return r


    def __get_count_list(self) -> dict[Song, int]:
        '''Return sorted dict representing # of times a Song was mentioned in responses for all Songs in the collection'''
        flist = {}
        for s in self.songs:
            flist[s] = 0
        for u in self.responses['unified']:
            for s in u:
                if s == 'nan' or s not in self.songs:
                    continue
                flist[s] += 1
        return {k: v for k, v in sorted(flist.items(), key=lambda item: item[1])}

    def __get_song_count_list(self, songmatch) -> dict[Song, int]:
        '''Return sorted dict of the count of other songs in the responses for a given Song (det. by songmatch)'''
        song = self.get(songmatch)

        flist = {}
        for s in self.songs:
            flist[s] = 0
        mask = self.responses['unified'].apply(lambda l: song.name in l)
        entries = self.responses[mask]['unified']
        for u in entries:
            for s in u:
                if s == 'nan' or s not in self.songs:
                    continue
                flist[s] += 1

        return {k: v for k, v in sorted(flist.items(), key=lambda item: item[1])}

    def __get_song_inbound_percent_list(self, songmatch, dig=2) -> dict[Song, float]:
        '''
        Return the inbound percents for a song. 
        i.e. for each Song s in a given Song t's count list, divide s's value in t's count list by the count of s in the responses
        This gives us a way to see the percent of s's listeners that listen to t.
        '''
        song = self.get(songmatch)
        flist = self.song_count_list[song]
        root_flist = self.count_list
        for s in flist:
            flist[s] /= root_flist[s]
            flist[s] = round(flist[s], dig)
        return {k: v for k, v in sorted(flist.items(), key=lambda item: item[1])}

    def __get_song_outbound_percent_list(self, songmatch, dig=2) -> dict[Song, float]:
        '''
        Return the outbound percents for a song
        i.e. for each Song s in a given Song t's count list, divide s's value in t's count list by the count of t in the responses
        This gives us a way to see the percent of t's listeners that listen to s.
        '''
        song = self.get(songmatch)
        flist = self.song_count_list[song]
        root_listens = self.count_list[song]
        for s in flist:
            flist[s] /= root_listens
            flist[s] = round(flist[s], dig)
        return {k: v for k, v in sorted(flist.items(), key=lambda item: item[1])}
        
    
    



In [47]:
songs = [Song(name) for name in unique_songs]
with open('songs', 'w') as f:
    f.write(jsons.dumps(songs))

In [35]:
# with open('songs', 'r') as f:
#     songs = jsons.loads(f.read(), List[Song])

In [48]:
col = SongCollection(responses=df, songs=songs)


In [61]:
for x in col.count_list.keys():
    print(type(x))
    break

<class '__main__.Song'>


In [68]:
with open('songcol.json', 'w') as f:
    f.write(jsons.dumps(col))

In [ ]:
with open('songcol.json', 'r') as f:
    col_ex = jsons.loads(f.read(), SongCollection)

## Update
Now, we have inbound data (x% of Bad Boy listeners listen to Aitai-tai). What about reverse? (Aitai-tai listeners listen to _).
Note that it's not necessary that there is a 1:1 relationship (Song A might have high chance to listen to song B, but there are a certain amount of song B listeners that don't like song A).

In [13]:
# this gives us what the update is talking about.
li = col.get_song_inbound_percent_list('Aitai-tai')

In [14]:
# % of Aitai-tai listeners that listen to Bad Boy
# li['Bad Boy']
li

{Psycho - Red Velvet - None: 0.2,
 Bad Boy - Red Velvet - None: 0.21,
 Peek-A-Boo - Red Velvet - None: 0.23,
 Monster - Red Velvet - None: 0.24,
 Sunny Side Up! - Red Velvet - None: 0.24,
 Kingdom Come - Red Velvet - None: 0.24,
 Red Flavor - Red Velvet - None: 0.25,
 Russian Roulette - Red Velvet - None: 0.25,
 In & Out - Red Velvet - None: 0.25,
 Power Up - Red Velvet - None: 0.27,
 Zimzalabim - Red Velvet - None: 0.27,
 Dumb Dumb - Red Velvet - None: 0.27,
 Naughty - Red Velvet - None: 0.27,
 You Better Know - Red Velvet - None: 0.27,
 Automatic - Red Velvet - None: 0.28,
 One of These Nights - Red Velvet - None: 0.28,
 Umpah Umpah - Red Velvet - None: 0.28,
 Ice Cream Cake - Red Velvet - None: 0.28,
 RBB - Red Velvet - None: 0.28,
 Happiness - Red Velvet - None: 0.29,
 Be Natural - Red Velvet - None: 0.3,
 Remember Forever - Red Velvet - None: 0.3,
 Eyes Locked Hands Locked - Red Velvet - None: 0.3,
 La Rouge - Red Velvet - None: 0.3,
 I Just - Red Velvet - None: 0.3,
 Love Is The 

## Graphing, Network Analysis

### Questions on dataset
* Which songs have high mutual connectivity?

In [ ]:
class CollectionStatistics:
    def __init__(self, col: SongCollection):
        self.col = col
    
    def get_mutual_pairs(self, cutoff=0.5, attr=None, attr_filter=None):
        '''Return all edges that satisfy the '''
